In [1]:
import dataset 
import tensorflow as tf
import time
from datetime import timedelta
import math
import random
import numpy as np
import time
import pickle
from pathlib import Path
import math

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from  numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
print(tf.__version__)
tf.get_default_graph()

1.8.0


load du lieu

In [3]:
# khoi tao gia tri load du lieu

train_path= r"D:\Hoang\data_test"

classes = dataset.get_labels(train_path)

print(classes)

num_classes = len(classes)
print(num_classes)

['a_h', 'a_k', 'chi_h', 'chi_k', 'e_h', 'e_k', 'i_h', 'i_k']
8


In [4]:
#load du lieu 
        
validation_size = 0.1
test_size = 0.2
img_size = 64
num_input_channels = 1

class DataCompression(object):
    def __init__(self,data):
        self.train = data.train
        self.valid = data.valid
        self.test = data.test

p=Path(r"D:\Hoang\train_data_experiment.file")
 
if not p.exists():
    print("Get and Compress data")
    data_train=dataset.read_train_sets(train_path=train_path,image_size=img_size,test_size=test_size,validation_size=validation_size)
    data_train = DataCompression(data_train)
    
    with open(p, "wb") as f:
        pickle.dump(data_train, f, pickle.HIGHEST_PROTOCOL)
    
#read data from file data compression
with open(p,"rb") as f:
    print("uncompress data")
    data = pickle.load(f)
    


uncompress data


In [5]:
print(data.train.images.shape)
print(data.valid.images.shape)
print(data.test.images.shape)

(76995, 64, 64, 1)
(10999, 64, 64, 1)
(21998, 64, 64, 1)


# Xây dựng hàm cho CNN

Khởi tạo giá trị

In [6]:
tf.reset_default_graph()

with tf.name_scope("inputs"):
    x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_input_channels], name='x')

    # labels
    y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

    keep_prob=tf.placeholder(tf.float32,name="keep_prob")
    
    y_true_cls = tf.argmax(y_true, axis=1)

In [7]:
# cau truc CNN M6-1

# filter_size_conv1 = 3 
# num_filters_conv1 = 32

# filter_size_conv2 = 3
# num_filters_conv2 = 32

# filter_size_conv3 = 3
# num_filters_conv3 = 64

# filter_size_conv4 = 3
# num_filters_conv4 = 64
    
# fc_layer_size = 256

#cau truc CNN 
filter_size_conv1 = 3 
num_filters_conv1 = 32

filter_size_conv2 = 3
num_filters_conv2 = 64

fc_layer_size = 512

In [8]:
def create_weights(shape,name):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05),name=name)

def create_biases(size,name):
    return tf.Variable(tf.constant(0.05, shape=[size]),name=name)

In [9]:
def create_conv_layer(input,
               num_input_channels, # so channels của từng ảnh
               filter_size,    # kích thước filter, kenel, 
               num_filters,  # số lượng đầu ra cho layer sau
               use_maxpool=False, #sử dụng max_pooling k?
               name = "conv_x" ):  
    with tf.name_scope(name):
        #tao weight và bias cho lop convolutional nay/
#         print("input {}: {}".format(name,input))

        name_weights = name+"_weights"
        with tf.name_scope("weights"):
            weights = create_weights(shape=[filter_size, filter_size, num_input_channels, num_filters],name=name_weights)
#         print("weights {}: {}".format(name,weights))

        name_biases = name+"_biases"
        with tf.name_scope("biases"):
            biases = create_biases(num_filters,name=name_biases)
        ## xay dung convolution layer, layer trả về có dimension nhu input
#         with tf.device('/gpu:0'):
        layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')
#         print("layer {}: {}".format(name,layer))

        layer += biases
        layer = tf.nn.relu(layer)

        if use_maxpool:
            layer = tf.nn.max_pool(value=layer,
                                    ksize=[1, 2, 2, 1],
                                    strides=[1, 2, 2, 1],
                                    padding='SAME')


#         store in histogram
        tf.summary.histogram("weights",weights)
        tf.summary.histogram("biases",biases)
        tf.summary.histogram("layer",layer)
        
        
        layer = tf.identity(layer,name=name)
        return layer

def create_flatten_layer(layer,use_dropout, name="flatten"):
    with tf.name_scope(name):
        # chuyen mỗi ảnh thanh mot vector, moi anh sẽ có kích thước phẳng là: img_size * img_size *num_channels

        #dimension layer: batch_size,img_size,img_size,num_channels 

        #lấy số lượng data
        layer_shape = layer.get_shape()

        # số lượng feature mỗi ảnh = img_size * img_size *num_channels
        num_features = layer_shape[1:4].num_elements()

        # reshape lại data ví dụ (10,2,2,5) // có 10 ảnh và mỗi ảnh có 2*2*5=20 feature => flat_layer: (10,20) 
        layer = tf.reshape(layer, [-1, num_features])
#         print("layer {}: {}".format(name,layer))
        layer = tf.identity(layer,name=name) # set name for layer
        if use_dropout:
             layer=tf.nn.dropout(layer,keep_prob);
        return layer

def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=False, 
             use_dropout=False,
             name = "fc"):
    
    with tf.name_scope(name):
        name_weights = name+"_weights"
        name_biases = name+"_biases"
        with tf.name_scope("weights"):
            weights = create_weights(shape=[num_inputs, num_outputs],name=name_weights)
        with tf.name_scope("biases"):
            biases = create_biases(num_outputs,name=name_biases)
    
#         with tf.device('/gpu:0'):
        layer = tf.matmul(input, weights) + biases

        if use_relu:
            layer = tf.nn.relu(layer)
        if use_dropout:
             layer=tf.nn.dropout(layer,keep_prob);
#         print("layer {}: {}".format(name,layer))

        tf.summary.histogram("weights",weights)
        tf.summary.histogram("biases",biases)
        tf.summary.histogram("layer",layer)
        
        layer = tf.identity(layer,name=name)
        return layer

# Xây dựng cấu trúc CNN
conv3-32 -> conv3-32 ->maxpool -> conv3-64 -> conv3-64 -> maxpool ->fc-256 dropout->fc10

In [10]:
# conv1 =  create_conv_layer(input=x,num_input_channels=num_input_channels,
#                            filter_size=filter_size_conv1,num_filters=num_filters_conv1,use_maxpool=False, name="conv1")

# conv2 = create_conv_layer(input=conv1,num_input_channels=num_filters_conv1,
#                           filter_size=filter_size_conv2,num_filters=num_filters_conv2,use_maxpool=True, name="conv2")

# conv3 = create_conv_layer(input=conv2,num_input_channels=num_filters_conv2,
#                           filter_size=filter_size_conv3,num_filters=num_filters_conv3,use_maxpool=False, name="conv3")

# conv4 = create_conv_layer(input=conv3,num_input_channels=num_filters_conv3,
#                           filter_size=filter_size_conv4,num_filters=num_filters_conv4,use_maxpool=True, name="conv4")

# layer_flat = create_flatten_layer(conv4, use_dropout=True)

# num_feature = layer_flat.get_shape()[1:2].num_elements()

# fc1 = create_fc_layer(input=layer_flat,num_inputs=num_feature,num_outputs=fc_layer_size,use_relu=True,use_dropout=True,name="fc1")


# fc2 = create_fc_layer(input=fc1,num_inputs=fc_layer_size,num_outputs=num_classes,use_relu=False,use_dropout=False,name="fc2")



In [11]:
## My model conv3-32 ->maxpool -> conv3-64->maxpool -> flatten dropout-> fc512 dropout  ->fc10

conv1 =  create_conv_layer(input=x,num_input_channels=num_input_channels,
                           filter_size=filter_size_conv1,num_filters=num_filters_conv1,use_maxpool=True, name="conv1")

conv2 = create_conv_layer(input=conv1,num_input_channels=num_filters_conv1,
                          filter_size=filter_size_conv2,num_filters=num_filters_conv2,use_maxpool=True, name="conv2")

layer_flat = create_flatten_layer(conv2, use_dropout=True)

num_feature = layer_flat.get_shape()[1:2].num_elements()
fc1 = create_fc_layer(input=layer_flat,num_inputs=num_feature,num_outputs=fc_layer_size,use_relu=True,use_dropout=True,name="fc1")


fc2 = create_fc_layer(input=fc1,num_inputs=fc_layer_size,num_outputs=num_classes,use_relu=False,use_dropout=False,name="fc2")



predict

In [12]:
with tf.name_scope("Train"):
#     with tf.device('/gpu:0'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc2,
                                                        labels=y_true)
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(cross_entropy)
        tf.summary.scalar("loss",loss)
    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)


with tf.name_scope("Test"):
    y_pred = tf.nn.softmax(fc2,name='y_pred')
    y_pred_cls = tf.argmax(y_pred, axis=1)
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
    
    tf.summary.scalar("accuracy",accuracy)



In [13]:
my_config =tf.ConfigProto(allow_soft_placement=True,log_device_placement=True)
my_config.gpu_options.allow_growth = True
session = tf.Session(config=my_config)

session.run(tf.global_variables_initializer())

saver = tf.train.Saver()

merged =tf.summary.merge_all();
train_writer= tf.summary.FileWriter(r"..\a_model_khoa_luan\graph\train",session.graph)
valid_writer=tf.summary.FileWriter(r"..\a_model_khoa_luan\graph\valid",session.graph)

In [14]:
def train(num_epochs, batch_size):
    num_iteration = int(data.train.images.shape[0]/batch_size)

    print_loss_iteration = int(num_iteration /5);
    total_iterations =0
    for epoch in range(num_epochs):
        for i in range(num_iteration):
            print(f"i: {i}")
            x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
    #         print(y_true_batch.shape)
            feed_dict_train = {x: x_batch, y_true: y_true_batch, keep_prob: 0.5}    

            session.run(train_step, feed_dict=feed_dict_train )
            
            if (i % print_loss_iteration == 0 or i==num_iteration-1):
                
                x_valid_batch, y_valid_batch, iname_valid_batch,cls_valid_batch = data.train.next_batch(500)
                
                feed_dict_valid = {x: x_valid_batch, y_true: y_valid_batch, keep_prob:1}
                feed_dict_train = {x: x_batch, y_true: y_true_batch, keep_prob: 1}

                valid_loss,valid_accuracy = session.run([loss,accuracy], feed_dict=feed_dict_valid)
 
                train_loss,train_accuracy = session.run([loss,accuracy], feed_dict=feed_dict_train)
                
                total_iterations=total_iterations+print_loss_iteration

                
                print(f"epoch: {epoch} -- iteration: {total_iterations} -- train_loss: {train_loss:.2f} -- train_acc {train_accuracy:.2%} --valid_loss:{valid_loss:.2f} -- valid_acc {valid_accuracy:.2%} -- time: {time.ctime()}")
#                 if i==num_iteration-1:
#                     print(f"save graph -- epoch: {epoch}-- iteration: {i}")
                train_merged=session.run(merged, feed_dict=feed_dict_train)
                valid_merged = session.run(merged, feed_dict=feed_dict_valid)
                train_writer.add_summary(train_merged,total_iterations)
                valid_writer.add_summary(valid_merged,total_iterations)
            
            
        save_path=saver.save(session, r'..\a_model_khoa_luan\model\modelTest.ckpt',global_step=epoch)

def accuracy_test():
    
    num_interations = int(data.test.images.shape[0]/500)
    
    for iteration in range(num_interations):
        x_batch_test, y_true_batch_test, _, cls_batch_test = data.test.next_batch(500)
        feed_dict_test = {x:x_batch,y_true:y_batch,keep_prob:1.0}
        acc=session.run(accuracy,feed_dict =feed_dict_test)
        print(f"iter: {iteration} -- accuracy test : {acc}")

In [15]:
start_time = time.time()
print(f"start time: {time.ctime(start_time)}")
train(num_epochs=3, batch_size =100)
end_time= time.time()
print(f"end time: {time.ctime(start_time)}")
run_time =(end_time-start_time)/60
print(f"run time: {run_time} m ")

start time: Sat Aug 11 22:32:51 2018
i: 0
epoch: 0 -- iteration: 153 -- train_loss: 2.05 -- train_acc 17.00% --valid_loss:2.07 -- valid_acc 14.80% -- time: Sat Aug 11 22:32:55 2018
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
i: 20
i: 21
i: 22
i: 23
i: 24
i: 25
i: 26
i: 27
i: 28
i: 29
i: 30
i: 31
i: 32
i: 33
i: 34
i: 35
i: 36
i: 37
i: 38
i: 39
i: 40
i: 41
i: 42
i: 43
i: 44
i: 45
i: 46
i: 47
i: 48
i: 49
i: 50
i: 51
i: 52
i: 53
i: 54
i: 55
i: 56
i: 57
i: 58
i: 59
i: 60
i: 61
i: 62
i: 63
i: 64
i: 65
i: 66
i: 67
i: 68
i: 69
i: 70
i: 71
i: 72
i: 73
i: 74
i: 75
i: 76
i: 77
i: 78
i: 79
i: 80
i: 81
i: 82
i: 83
i: 84
i: 85
i: 86
i: 87
i: 88
i: 89
i: 90
i: 91
i: 92
i: 93
i: 94
i: 95
i: 96
i: 97
i: 98
i: 99
i: 100
i: 101
i: 102
i: 103
i: 104
i: 105
i: 106
i: 107
i: 108
i: 109
i: 110
i: 111
i: 112
i: 113
i: 114
i: 115
i: 116
i: 117
i: 118
i: 119
i: 120
i: 121
i: 122
i: 123
i: 124
i: 125
i: 126
i: 127
i: 128
i: 129
i: 130
i: 131
i: 132
i: 

KeyboardInterrupt: 

In [ ]:
accuracy_test()

In [ ]:
80*64*64/100